# 6 RAG

* 인덱싱
  데이터 로딩

  데이터를 작은 단위의 청크로 나눔

  임베딩 > 수치화 > 3차원 배열 속 저장

  임베딩 > 벡터 DB

* 검색 & 생성

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='../.env', override=True)

api = os.getenv("API2")
default_model = os.getenv("OPENAI_DEFAULT_MODEL")

### Doucument Loader

파일
* TextLoader
| 일반 텍스트 파일(.txt) 읽기
* PDFPlumberLoader / PyPDFLoader
| PDF 문서 불러오기
* UnstructuredFileLoader
| 다양한 파일 포맷 (Word, PPT, PDF 등)을 구조 없이 불러오기
* CSVLoader
| CSV 파일의 각 행을 문서로 변환
* JSONLoader
| JSON 파일을 읽고 문서로 구성


웹/URL
* WebBaseLoader
| 일반 웹 페이지 크롤링
* SitemapLoader
| 사이트맵 기반으로 다수의 웹 페이지 로딩


문서 플랫폼
* NotionDBLoader
| Notion의 DB에서 문서 불러오기 (API 필요)
* ConfluenceLoader
| Atlassian Confluence 문서 불러오기


클라우드 문서
* GoogleDriveLoader
| 구글 드라이브에서 문서 불러오기
* OneDriveLoader
| 마이크로소프트 OneDrive에서 불러오기


코드/로컬
* GitLoader
| Git 저장소 내 파일 불러오기
* DirectoryLoader
| 특정 폴더 내 모든 문서 일괄 불러오기


메일
* OutlookMessageLoader
| Outlook .msg 파일 불러오기
* EmailLoader
| EML 또는 MIME 형식 이메일 로드


이미지/스캔
* UnstructuredImageLoader
| 이미지에서 텍스트 추출 (OCR 기반)
* UnstructuredPDFLoader
| PDF 전처리 고급

//////////////////////////////////
#### 텍스트 불러오기

In [ ]:
from langchain.document_loaders import TextLoader, PDFPlumberLoader, WebBaseLoader

# 텍스트 파일
text_loader = TextLoader("..\\data\\data.txt", encoding="utf-8")
docs_text = text_loader.load()
print(docs_text)

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': '..\\data\\data.txt'}, page_content='프랑스의 수도는 파리이다. 파리는 유럽에서 가장 인기 있는 관광 도시 중 하나로, 에펠탑과 루브르 박물관이 유명하다.\n\n독일의 수도는 베를린이다. 베를린은 역사적으로 중요한 도시이며, 베를린 장벽으로 유명하다.\n\n일본의 수도는 도쿄이다. 도쿄는 기술과 문화의 중심지로, 애니메이션과 음식 문화가 발달해 있다.')]


#### 청크로 나누기

청크 사이즈는 RAG의 성능을 결정하는 하이퍼파라미터가 됨 

In [3]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=5)

docs =text_splitter.split_documents(docs_text)
print(len(docs))
print(docs)

3
[Document(metadata={'source': '..\\data\\data.txt'}, page_content='프랑스의 수도는 파리이다. 파리는 유럽에서 가장 인기 있는 관광 도시 중 하나로, 에펠탑과 루브르 박물관이 유명하다.'), Document(metadata={'source': '..\\data\\data.txt'}, page_content='독일의 수도는 베를린이다. 베를린은 역사적으로 중요한 도시이며, 베를린 장벽으로 유명하다.'), Document(metadata={'source': '..\\data\\data.txt'}, page_content='일본의 수도는 도쿄이다. 도쿄는 기술과 문화의 중심지로, 애니메이션과 음식 문화가 발달해 있다.')]


#### 임베딩

* openai 임베딩에 사용 가능한 모델

  * text-embedding-3-small
  * text-embedding-3-large
  * text-embedding-ada-002

In [3]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=api)

NameError: name 'api' is not defined

In [2]:
ex_text = "프랑스의 수도는 파리이다. 파리는 유럽에서 가장 인기 있는 관광 도시 중 하나로, 에펠탑과 루브르 박물관이 유명하다."
vector = embeddings.embed_query(ex_text)

NameError: name 'embeddings' is not defined

In [1]:
doc_texts = [doc.page_content for doc in docs]

vectors = embeddings.embed_documents(doc_texts)

NameError: name 'docs' is not defined

### ChromaDB 생성하기

In [7]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="..\\chroma_db"
)
vectorstore.persist()

C:\Users\JS\AppData\Local\Temp\ipykernel_21856\503007652.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


# Retrieval & LLM 생성

In [8]:
from langchain.chat_models import ChatOpenAI

retriever = vectorstore.as_retriever()

llm = ChatOpenAI(api_key=api,
                 model=default_model,
                 temperature=0)

C:\Users\JS\AppData\Local\Temp\ipykernel_21856\22402204.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(api_key=api,


In [9]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

### 질문하기

In [13]:
query = "프랑스의 수도는 무엇인가요?"
result = qa_chain.run(query)
print(result)

프랑스의 수도는 파리입니다.


* 프롬프트를 함께 제공하는 법

In [15]:
query = "영국의 수도는 무엇인가요?"

ret = retriever.get_relevant_documents(query)

if not ret:
    print("검색 결과가 없습니다.")
else:
    result = qa_chain.run(query)
    print(result)

C:\Users\JS\AppData\Local\Temp\ipykernel_21856\3092354485.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ret = retriever.get_relevant_documents(query)


죄송하지만, 제공된 정보에는 영국의 수도에 대한 내용이 없습니다.


In [17]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    너는 주어진 context(문서)에서만 답변해야 해. 만약 context가 비어 있거나, 
    context에 답이 없으면 반드시 "그런 내용 없습니다."라고 답해.
    Context: {context} 
    Question: {question}
    답변:"""
)

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                       retriever=retriever,
                                       chain_type_kwargs={"prompt": custom_prompt},
                                       return_source_documents=False)

In [19]:
query = "영국의 수도는 무엇인가요?"
result = qa_chain.run(query)
print(result)

그런 내용 없습니다.


In [21]:
sentence1 = "안녕하세요? 반갑습니다."

sentence2 = "안녕하세요? 반갑습니다!"

sentence3 = "안녕하세요? 만나서 반가워요."

sentence4 = "Hi, nice to meet you."

sentence5 = "I like to eat apples."

In [27]:
from langchain.embeddings import OpenAIEmbeddings
import numpy as np
from itertools import combinations

#1. 임베딩 생성기 준비
embeddings = OpenAIEmbeddings()

#2. 문장 리스트
sentences = [
    "안녕하세요? 반갑습니다.",
    "안녕하세요? 반갑습니다!",
    "안녕하세요? 만나서 반가워요.",
    "Hi, nice to meet you.",
    "I like to eat apples.",
    "아침 못먹었어요",
    "吃羊肉串的时候, 我超喜欢喝啤酒",
    "Abi in malam crucem!",
    "ちょっとまって",
    "你好"
]

#3. 각 문장을 임베딩 벡터로 변환
sentence_vectors = [embeddings.embed_query(sentence) for sentence in sentences]

#4. 코사인 유사도 함수 정의
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

#5. 모든 문장 쌍에 대해 코사인 유사도 계산 및 출력
print("문장 쌍별 코사인 유사도:")
for (i, j) in combinations(range(len(sentences)), 2):
    sim = cosine_similarity(sentence_vectors[i], sentence_vectors[j])
    print(f"[{i+1}] \"{sentences[i]}\" <-> [{j+1}] \"{sentences[j]}\" : {sim:.4f}")

문장 쌍별 코사인 유사도:
[1] "안녕하세요? 반갑습니다." <-> [2] "안녕하세요? 반갑습니다!" : 0.9881
[1] "안녕하세요? 반갑습니다." <-> [3] "안녕하세요? 만나서 반가워요." : 0.9463
[1] "안녕하세요? 반갑습니다." <-> [4] "Hi, nice to meet you." : 0.8448
[1] "안녕하세요? 반갑습니다." <-> [5] "I like to eat apples." : 0.7296
[1] "안녕하세요? 반갑습니다." <-> [6] "아침 못먹었어요" : 0.8169
[1] "안녕하세요? 반갑습니다." <-> [7] "吃羊肉串的时候, 我超喜欢喝啤酒" : 0.7213
[1] "안녕하세요? 반갑습니다." <-> [8] "Abi in malam crucem!" : 0.7151
[1] "안녕하세요? 반갑습니다." <-> [9] "ちょっとまって" : 0.7575
[1] "안녕하세요? 반갑습니다." <-> [10] "你好" : 0.8256
[2] "안녕하세요? 반갑습니다!" <-> [3] "안녕하세요? 만나서 반가워요." : 0.9379
[2] "안녕하세요? 반갑습니다!" <-> [4] "Hi, nice to meet you." : 0.8360
[2] "안녕하세요? 반갑습니다!" <-> [5] "I like to eat apples." : 0.7266
[2] "안녕하세요? 반갑습니다!" <-> [6] "아침 못먹었어요" : 0.8118
[2] "안녕하세요? 반갑습니다!" <-> [7] "吃羊肉串的时候, 我超喜欢喝啤酒" : 0.7183
[2] "안녕하세요? 반갑습니다!" <-> [8] "Abi in malam crucem!" : 0.7315
[2] "안녕하세요? 반갑습니다!" <-> [9] "ちょっとまって" : 0.7539
[2] "안녕하세요? 반갑습니다!" <-> [10] "你好" : 0.8238
[3] "안녕하세요? 만나서 반가워요." <-> [4] "Hi, nice to meet you." : 0.8372
[3] 